In [9]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from tqdm import tqdm
from scipy import ndimage

In [10]:
import torch.nn.functional as F
import torch as tc
from helpers_img import *
from NeuralNets import *
from training_NN import *
from Post_processing import *

In [ ]:
tc.cuda.empty_cache()

In [ ]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
N = min(100, len(files)) # Load maximum 100 images
print("Loading " + str(N) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(N)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(N) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(N)]
print(files[0])

#n = 85 # Only use 85 images for training

In [ ]:
# augment the dataset adding rotated images
imgs, gt_imgs = rotation(imgs, gt_imgs)
print('Total number of imgages: '+str(len(imgs)))

In [ ]:
del(root_dir)
del(image_dir)
del(files)
del(N)
del(gt_dir)

In [ ]:
def bootstrap_images(imgs, gt_imgs,size,number):
    '''This function returns a list of list. Each element of the "external" list is a list of randomly 
    sampled images with replacement'''
    
    new_imgs=[]
    new_gt_imgs=[]
    array = np.arange(len(imgs))
    matrix= np.zeros((len(imgs),number))
    for k in range(number):
        b = np.random.choice(array, size, replace=True)
        list_temp_imgs = [imgs[i] for i in b]
        list_temp_gt_imgs = [gt_imgs[i] for i in b]
        new_imgs.append(list_temp_imgs)
        new_gt_imgs.append(list_temp_gt_imgs)
        matrix[b,k]=1
    
    return new_imgs,new_gt_imgs,matrix

In [ ]:
def train_SimpleNet(dataset, label, w, h, lr, max_epochs, mini_batch_size, dropout):
    ''' Train a simple net'''
    n = len(dataset)
    train_sub_images = [img_crop(dataset[i], w, h) for i in range(n)]
    train_mask_label = [img_crop(label[i],w,h) for i in range(n)]
    train_mask_label = from_mask_to_vector(train_mask_label,0.3)
    train_sub_images = transform_subIMG_to_Tensor(train_sub_images)
    mean = train_sub_images.mean()
    std = train_sub_images.std()
    train_sub_images = (train_sub_images-mean)/std
    train_sub_images, train_mask_label = reduce_dataset(train_sub_images,train_mask_label)
    # shuffle images
    for l in range(10):
        new_indices= np.random.permutation(len(train_mask_label))
        train_sub_images=train_sub_images[new_indices]
        train_mask_label=train_mask_label[new_indices]
    
    model = SimpleNet(dropout)
    
    mini_batch_rest = train_sub_images.size(0) % mini_batch_size
    
    if mini_batch_rest > 0:
        train_sub_images = train_sub_images.narrow(0,0,train_sub_images.size(0)-mini_batch_rest)
        train_mask_label = train_mask_label[0:train_sub_images.size(0)]
        
        
        

    train_model_Adam( model, train_sub_images, train_mask_label, max_epochs, lr, mini_batch_size)
    
    return model



In [ ]:
def bagging_NN(dataset, label, percentage_train_data, nb_model, w, h, lr, max_epochs, mini_batch_size, dropout):
    nb_data = int( len(dataset)*percentage_train_data )
    list_dataset, list_label, data_matrix = bootstrap_images(dataset, label, nb_data, nb_model)
    models = []
    for i in range(nb_model):
        model=train_SimpleNet(list_dataset[i], list_label[i], w, h, lr, max_epochs, mini_batch_size, dropout)
        models.append(model)
        print('model '+str(i)+' trained')
        
    data_matrix = 1 - data_matrix
    # the data matrix has 1 in position n,j if the nth image was not used in jth
    # model training.
    
    # compute F1 error
    
    test_imgs=[img_crop(dataset[k], w, h) for k in range(len(dataset))]
    nb_patches=len(test_imgs[0])
    test_imgs = transform_subIMG_to_Tensor(test_imgs)
    mean=test_imgs.mean()
    std= test_imgs.std()
    test_imgs = (test_imgs-mean)/std
    F1_error=0
    not_testable_img=0
    for i in range(len(dataset)):
        image= test_imgs.narrow(0,i*nb_patches,nb_patches)
        if data_matrix[i,:].sum()>0:
            ind=np.where(data_matrix[i,:])[0]
            predictions=[models[k](image).detach().numpy() for k in ind]
            predictions = np.array(predictions)
            predictions = ((predictions.mean(0)[:] >0.5)*1).reshape(-1,)
            mask_test = label_to_img(400, 400, w, h, predictions)
            F1_error += calcul_F1(label[i], mask_test)
        else:
            not_testable_img+=1
    
    F1_error= F1_error/(len(dataset)-not_testable_img)
    return models, F1_error
    

In [ ]:
def train_model_Adam( model, train_data, label, max_epochs, lr, mini_batch_size, threshold=0.01):
    '''train the Neural Net using Adam as optimizer and an MSE loss'''
    #optimizer=tc.optim.SGD(model.parameters(),lr)
    optimizer=tc.optim.Adam(model.parameters(),lr)
    criterion= tc.nn.MSELoss()
    training_errors=[]
    if tc.cuda.is_available():
        tc.cuda.empty_cache()
        model.cuda()
        train_data = train_data.cuda()
    
    for epoch in tqdm(range(max_epochs)):
        model.is_training=True
        model.train()
        if tc.cuda.is_available():
            tc.cuda.empty_cache()
        for i in range(0,train_data.size(0),mini_batch_size):
            output= model(train_data.narrow(0,i,mini_batch_size))
            #print(output,tc.LongTensor(np.array([1*label[i:i+mini_batch_size]]).reshape(-1,1)))
            temp=tc.FloatTensor(np.array([1*label[i:i+mini_batch_size]]).reshape(-1,1))
            
            temp = temp.cuda()
            loss= criterion(output,temp)
            model.zero_grad()
            loss.backward()
            optimizer.step()
        # compute training error
        model.is_training=False
        model.eval()
        test = model(train_data)
        test = test.cpu()
        prediction= test[:]>0.5
        
        prediction= 1*(prediction.numpy()[:] != label.reshape(-1,1)[:])
        
        training_error = np.sum(prediction)/len(prediction)
        training_errors.append(training_error*100)
        if training_error< threshold:
            break
        
    
    plt.figure()
    plt.plot(np.arange(epoch+1)+1,training_errors)
    plt.xlabel('epoch')
    plt.ylabel('error [%]')
    plt.show()
        
    model.cpu()    
    

In [ ]:
# valerio modifica questo
percentage_train_data=0.1
nb_model=10
w=16
h=16
lr=1e-4
max_epochs=20
mini_batch_size=10
dropout=0
models, F1_error= bagging_NN(imgs, gt_imgs, percentage_train_data, nb_model, w, h, lr, max_epochs, mini_batch_size, dropout)

In [ ]:
print(F1_error)

In [ ]:
for i,model in enumerate(models):
    tc.save(model,f'Model_Bagging/model{i}.pt')

In [3]:
models=[]
for i in range(10):
    model = tc.load(f'Model_Bagging/model{i}.pt')
    models.append(model)

In [ ]:
test_img = imgs[30]
mask= gt_imgs[30]
test_img = [img_crop(test_img, 16,16)]
test_img = transform_subIMG_to_Tensor(test_img)
mean=test_img.mean()
std= test_img.std()
test_img = (test_img-mean)/std
result=np.zeros((test_img.size(0),))
for model in models:
    result += model(test_img).detach_().numpy().reshape(-1,)

result = result / len(models)
result = (result > 0.5)*1


image = imgs[30]

mask_res = label_to_img(400, 400, 16, 16, result)
image_plot = make_img_overlay(image, mask_res)
plt.figure(figsize=(10, 10))
plt.imshow(image_plot)
    

In [5]:
def print_image_test(test_image, models):
    ''' take a 608 * 608 image'''
    image_original = test_image
    test_image = img_crop(test_image,16,16)
    test_image = transform_subIMG_to_Tensor([test_image])
    mean=test_image.mean()
    std= test_image.std()
    test_image = (test_image-mean)/std
    predictions=[]
    for model in models:
        model.eval()
        predictions.append(model(test_image).detach().numpy().reshape(-1,))
    
    predictions = np.array(predictions).mean(0)
    predictions = predictions > 0.5
    # add there the postprocessing
    predictions = post_processing(predictions,27,8,3,3,38)
    ###############################################
    predictions=label_to_img(608, 608, 16, 16, predictions)
    
    image = make_img_overlay(image_original, predictions)
    
    plt.figure(figsize=(10,10))
    plt.imshow(image)

In [6]:
# Loaded a set of images
root_dir_test = "test_set_images/"
imgs_test = []
for l in range(1,51):
    dir_test = root_dir_test + 'test_'+str(l)+'/'
    files_test = os.listdir(dir_test)
    img_test = load_image(dir_test + files_test[0])
    imgs_test.append(img_test)

In [8]:
#print_image_test(imgs_test[16], models)

In [ ]:
def create_submission(test_data, models, w, h, name_file, prediction_training_dir, normalize=True):
    ''' the function takes as input the test data and the models used for prediction. 
    If a list of model is given, the prediction will be done with majority vote. 
    
    The function is written explicitly for prediction using SimpleNet model.
    
    test_data: list of images.
    
    models: list of models or single model
    
    w, h: width and high of the patches'''
    
    # from list to Tensor
    w_im, h_im,_ = test_data[0].shape
    test_data = [img_crop(test_data[k], w, h) for k in range(len(test_data))]
    
    test_data = transform_subIMG_to_Tensor(test_data)
    
    if normalize:
        
        test_data = (test_data-test_data.mean())/test_data.std()
    
    try :
        nb_models = len(models)
        prediction = [] 
        for i in range(nb_models):
            models[i].eval()
            prediction.append((models[i](test_data)).detach().numpy())
            
        prediction = np.array(prediction)
        prediction = (prediction.sum(0)/nb_models > 0.5)*1
    
    except:
        
        prediction = models(test_data).detach().numpy()
        
        prediction = 1*(prediction > 0.5)
     
    prediction = prediction.reshape(-1,)
    
    nb_patches = int(w_im*h_im/(w*h))
    nb_images =int(prediction.shape[0]/nb_patches)
    list_of_mask = [prediction[i*nb_patches:(i+1)*nb_patches ] for i in range(nb_images)]
    for k in range(len(list_of_mask)):
        list_of_mask[k] = post_processing(list_of_mask[k],27,8,3,3,38)
    # from patch to image
    list_of_masks=[label_to_img(w_im, h_im, w, h, list_of_mask[k]) for k in range(nb_images)]
    list_of_string_names = []
    for i,gt_image in enumerate(list_of_masks):
        plt.imsave(prediction_training_dir+f"prediction_{i+1}.png",gt_image, cmap=matplotlib.cm.gray)
        list_of_string_names.append(prediction_training_dir+"prediction_" + str(i+1) + ".png")
    
    # create file submission
    masks_to_submission(name_file, *list_of_string_names)

In [ ]:
create_submission(imgs_test, models, w, h, 'first_submission.csv', '', normalize=True)